In [1]:
!pip install faiss-cpu
!pip install -U langchain_community
!pip install langchain transformers accelerate bitsandbytes huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/

importing libraries

In [2]:
import faiss
import pickle
import torch
import numpy as np
from torch import no_grad
from huggingface_hub import login
from google.colab import drive
from sentence_transformers import SentenceTransformer
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


loading the embedded data

In [4]:
index = faiss.read_index("/content/drive/MyDrive/complaint_index.faiss")

# Load chunks and metadata
with open("/content/drive/MyDrive/chunk_texts.pkl", "rb") as f:
    texts = pickle.load(f)

with open("/content/drive/MyDrive/chunk_metadata.pkl", "rb") as f:
    metadata = pickle.load(f)

Prepare the Retriever

In [5]:
model = SentenceTransformer("all-MiniLM-L6-v2", device="cuda")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

User Query + Embedding

In [6]:
query = input()
query_vec = model.encode([query])

what is the most asked question?


Retrieve Top-5 Chunks

In [7]:
top_k = 5
D, I = index.search(np.array(query_vec), top_k)

retrieved_chunks = [texts[i] for i in I[0]]
retrieved_metadata = [metadata[i] for i in I[0]]

Format Context for LLM

In [8]:
context = "\n\n".join(retrieved_chunks)
prompt = f"""
You are an AI assistant that summarizes customer complaints.

User question:
{query}

Based on these complaints:
{context}

Answer the question with specific insights from the text.
"""

Generate Answer Using OpenAI

In [10]:
torch.cuda.empty_cache()
login(new_session=False)
pipe = pipeline("text-generation", model="mistralai/Mistral-7B-Instruct-v0.1")

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Device set to use cuda:0


In [13]:
messages = [
    {"role": "user", "content": prompt},
]
output = pipe(messages)
print(output[0]['generated_text'][1]['content'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Based on the provided complaints, the most asked question appears to be related to a survey or inquiry that the user did not recognize or understand. The text also mentions that this inquiry was requested even though the user had already answered all of the questions before. Additionally, the user mentions that they had inquired on specific topics, such as xxxxxxxx, xxxx xxxxxxxx, and xxxx xxxxxxxx. It seems that the most common complaint among these complaints is related to confusion or frustration with an inquiry that the user did not understand or was unable to answer.
